# **Map Parts PostProcess**

## **1 New Clustering Sorting**
나머지 지역 필터링 2개 구역 22명: 3X7 + 4X4 = 37  

In [1]:
def table_sorting_function(group_number):
    # Cluster 단위로 순위 새롭게 정의하기
    tableA = table[table['group']==group_number].copy()

    # cluster 단위로 평균값 계산
    tableA_cluster = [[no+1, tableA[tableA['cluster']==_]['lon'].mean(), tableA[tableA['cluster']==_]['lat'].mean()]  
     for no, _  in enumerate(tableA['cluster'].unique())]
    tableA_cluster = pd.DataFrame(tableA_cluster).sort_values([1,2]).reset_index(drop=True)
    tableA_cluster = tableA_cluster.reset_index()
    tableA_cluster['index'] = list(map(lambda x : x+1, tableA_cluster['index']))

    # cluster 변환 원본과 대상 계산완료 및 적용
    tableA_cluster = {tableA_cluster.iloc[_,0] : tableA_cluster.iloc[_,1]  for _ in range(len(tableA_cluster))}
    print(tableA_cluster)
    tableA['cluster'] = [tableA_cluster[_] for _ in tableA['cluster']]

    # cluster 단위로 Store Index 새롭게 추가하기
    tableA = tableA.sort_values(['cluster', 'lon', 'lat']).reset_index(drop=True)

    store_index = []
    for _ in tableA['cluster'].unique():
        store_index += [_+1 for _ in range(tableA[tableA['cluster']==_].shape[0])]
    tableA['store'] = store_index
    return tableA

In [2]:
file = 'data/stores_clust_fix.csv'

import pandas as pd
# table = pd.read_csv(file).fillna('')
# table = [table_sorting_function(_)  for _ in table['group'].unique()]
# tables = pd.concat(table).reset_index(drop=True)
# tables.to_csv('data/stores_clust_fix_v2.csv', index=None, encoding="ms949")

# loading the CSV
tables = pd.read_csv('data/stores_clust_fix_v2.csv', encoding='ms949').fillna('')
tables.tail(3)

,title,no,part,address,dong,name,phone,telephone,order,lon,lat,color,group,cluster,store
109,꿈이있는문구점,119-03-40156,문구점,남부순환로208길 17,청룡동,이경애,010-3615-8635,02-885-7374,"간판, 천정, 벽페인팅 등",126.943431,37.481217,darkgreen,3,11,1
110,수미분식,119-14-53453,음식점,남부순환로208길 15,청룡동,김명숙,010-6852-5289,02-878-4311,"간판, 어닝개선",126.943471,37.481373,darkgreen,3,11,2
111,제일공인중개사사무소,119-12-26684,부동산중개업,"쑥고개로13길 9, 101호",청룡동,김영길,010-5204-9451,02-886-8600,"윈도우 시트교체, 의자테이블",126.943782,37.480012,darkgreen,3,11,3


In [3]:
from data.arterior import MapFolium
maplot = MapFolium()
map_instance = maplot.plot_circle(tables, title='cluster', zoom=16)
map_instance
map_instance.save('data/map_v2.html')

## **2 Result Tables**
테이블 작업결과 추가 수정
```
c6-3 => c9-4
c10-3 => c7-4
c11-1 => c6-4
c11-2 => c6-3
c11-3 => c10-3
```

In [4]:
change_data = [
    [(3,6,3), (3,9,4)],
    [(3,10,3), (3,7,4)],
    [(3,11,1), (3,6,4)],
    [(3,11,2), (3,6,3)],
    [(3,11,3), (3,10,3)]
]

In [5]:
for _ in change_data:
    original = list(_[0])
    target   = list(_[1])

    for no, _table_instance in enumerate(tables.iloc[:,-3:].values.tolist()):
        if _table_instance == original:
            tables.iloc[no, -3:] = target
        
tables = tables.sort_values(['group', 'cluster', 'store']).reset_index(drop=True)
tables.to_excel('data/stores_clust.xlsx')

In [6]:
from data.arterior import MapFolium
maplot = MapFolium()
map_instance = maplot.plot_circle(tables, title='cluster', zoom=16)
map_instance
map_instance.save('data/map_v3.html')